# Lidar Processing Workflow
**Cara Piske, Graduate Program of Hydrologic Sciences, 2022; Advisor: Dr. Adrian Harpold**<br>
This code processes raw lidar point clouds in order to calculate snow depth using PDAL. <br>
Lidar data were provided by the Airborne Snow Observatory (ASO), the National Center for Airborne Laser Mapping (NCALM), and Watershed Sciences Inc. (WSI). <br>

The goal of this project is to process snow depth to the one-meter spatial scale while maintaining conservative under-canopy snow depth estimates. Therefore, little interpolation occurs under-canopy. NCALM and WSI flights were obtained through OpenTopography.

Start by importing necessary files

In [2]:
# import necessary files
import numpy as np 
import matplotlib.pyplot as plt
import json # where we will save the json files to run a pipeline
import os # for file management
import subprocess # allows us to run command line commands
import pdal # lidar processing package
import rasterio
import pyproj
from osgeo import gdal

import time
# packages to copy files
from pathlib import Path
import shutil
import glob

# packages to extract wkt from polygon
import shapefile
import pygeoif

# for parallel processing
import concurrent.futures
from multiprocessing import Pool
import csv

# See lidar_functions.py
import lidar_functions


Note that many functions are dependent on specific directory structures. See README

In [2]:
# # Set current working directory
# path = 'path/to/current/working/directory/'
# os.chdir(path)
# os.getcwd() # print to ensure we're in the right directory

In [3]:
json_base_path = 'path/to/JSON/files/' # set so that we can redefine json across operating systems

Applied

In [8]:
# Set current working directory
path = '/'
os.chdir(path)
os.getcwd() # print to ensure we're in the right directory

'G:\\'

In [9]:
json_base_path = 'piske_processing/PDAL_workflow/JSON/' # set so that we can redefine json across operating systems

# Pre-processing

There are two ways to process LAS files. An efficient way, depending on computing power, is through parallelization. This involves running processes on tiles by spreading out the computations to multiple cores. Ultimately, the rasterized product will combine all tiles. For some of our processing, we combined all tiles prior to rasterization within a pipeline. 

## Info
Get to know lidar file... <br>
In general, running PDAL python bindings can be difficult, so an easy workaround is to initiate a terminal command and run via subprocess. This is what we'll do for most of the processing. 

In [1]:
# # define input file
# input_lid = 'full/path/to/las/lidar.las'
# pdal_info_cmd = ['pdal','info',input_lid] # general info
# pdal_metadata_cmd = ['pdal','info',input_lid,'--metadata'] # full file metadata, including details crs
# subprocess.run(pdal_metadata_cmd)

Save results to a dictionary <br>
This will be useful below when renaming files based on specific metadata... sometimes results aren't printed above so this is also helpful to isolate outputs and print. 

In [2]:
# pdal_info_results = subprocess.run(pdal_metadata_cmd, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
# pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # create dict with metadata info
# pdal_info_dict # print results

## Writers Bounds
Set the extents of the final output raster file. Setting this prior to raster operations is beneficial because it presets pixel delineation, prevents major cropping/resampling, and allows raster operations to occur directly from the final product. 

Check extents of all files in a directory (in order to create bounds for rasterization below)

In [ ]:
lidar_folder = 'path/to/directory/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
full_paths = [lidar_folder +f for f in onlyfiles]

Use lidar_functions to extract the min and max extents of each file in a flight <br>
For our purposes, we compared extents of all flights and used the values from the flight with the most limited spatial extent. 

In [ ]:
# # Use lidar_functions to extract the min and max extents of each file in a flight
# extents = lidar_functions.check_flight_extent(full_paths)

In [ ]:
# writers_bounds = '([llx, urx], [lly, ury])'

Applied

In [ ]:
# writers_bounds = '([730235.96, 738826.45], [4364741.66, 4372273.16])'

## Retile
Retiling lidar files allows for parallelization, speeding up processing. Standard tiles come in 500-1500m extents. Choose depending on point density and spatial extent of files. Many NCALM files come in 1000x1000m already. Buffer zones are dependent on point density etc. and allow for redundant classifications to avoid the edge effect.

**Single Folder**

In [1]:
# retile_input_path = 'path/to/input/lidar/folder/' # define directory to original LAS files
# retile_output_path = 'path/to/output/lidar/folder/' # define directory to output LAS tiles
# onlyfiles = [f for f in os.listdir(retile_input_path) if os.path.isfile(os.path.join(retile_input_path, f))] # create var list of every file in input directory
# for files in onlyfiles: # for each of these files
#     full_path = os.path.join(retile_input_path, files) # define the full file path
#     output_path = retile_output_path+'retile_#'+files # create new file path
#     retile_command = ['pdal', 'tile', full_path, output_path, '--length=1000','--buffer=50'] # define retile command to be run in the termine
#     subprocess.run(retile_command) # run the command

## Rename
Many files come with inconsistent naming (including unsupported characters...) <br>
Rename all files to maintain consistency. Here, our file structure is flight_watershed_date_llx_lly <br>
Note that some functions are written based on 

Start by renaming with llx and lly

In [ ]:
# # Many files come with inconsistent naming (including unsupported characters...)
# # Rename all files to maintain consistency

# # Rename a file with the lower left x and y defined as the corner point of the bounding box (add resolution/2 to get the center point of the box) 
# # input - full lidar file path (i.e. 'folder1/folder2/SCB/flight1/lid_files/filename.laz'), [str]

# def rename_llx_lly(full_path):
    
#     pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal info command
#     pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
#     pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dictionary
    
#     pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only pathname (i.e. 'folder1/folder2/SCB/flight1/lid_files/')
#     new_name = os.path.join(pathname, str(round(pdal_info_dict['metadata']['minx'])) +"_"+ str(round(pdal_info_dict['metadata']['miny']))+full_path[-4:])    
#     os.rename(full_path, new_name) # rename file
    

In [ ]:
# # run similar function except add in file tag. This is helpful for retiled files where there may be redundant llx_lly values 
# def rename_llx_lly_repeats(full_path):
    
#     pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal info command
#     pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
#     pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dictionary
#     pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only pathname (i.e. 'folder1/folder2/SCB/flight1/lid_files/')
#     new_name = os.path.join(pathname, str(round(pdal_info_dict['metadata']['minx'])) +"_"+ str(round(pdal_info_dict['metadata']['miny']))+full_path[-4:])    
#     if os.path.exists(new_name):
#         lidar_folder = pathname
#         onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#         full_paths = [os.path.join(lidar_folder, s) for s in onlyfiles] 
#         full_str = ','.join(full_paths)
#         num_occurences = full_str.count(new_name[:-4])
#         new_name_b = new_name[:-4]+'_'+str(num_occurences)+new_name[-4:]
#         os.rename(full_path, new_name_b)
#     else:
#         os.rename(full_path, new_name) # rename file

Add consistent string (FlightSource_Watershed_date) to the beginning of the filename

In [ ]:
# # See lidar_functions.py
# # rename a file with additional metadata at the beginning of the filename
# # flight organization, watershed, date of flight (i.e. ASO_ICB_20140423)
# # additional text will be taken from the flight directory name, could hardcode additional string instead of using folder name
# # input - full lidar file path (i.e. 'lidar/lidar_files/filename.laz'), str

# def add_str_to_filename(full_path):
#     filename = os.path.basename(full_path) # isolate only filename (i.e 'filename.laz')
#     pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only path name (i.e. 'lidar/lidar_files')
#     add_str = os.path.normpath(pathname) # split up the path name (i.e. full path)
#     add_str = [i for i in add_str.split(os.sep) if (i.startswith('ASO_') or i.startswith('NCALM_') or i.startswith('WSI_'))] # (i.e. 'ASO_SCB_2016')
#     rename = os.path.join(pathname, add_str[0] + '_'+ filename) # add string to full pathname 
#     os.rename(full_path, rename) # rename file

**Single file**

In [ ]:
# rename_llx_lly('filepath.laz')
# add_str_to_filename('new_filepath.laz')

**Parallel Processing**

In [ ]:
# lidar_folder = 'path/to/lid/folder/'
# tic = time.perf_counter()
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#         full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
#         executor.map(lidar_functions.rename_llx_lly, full_path) #running 10 times
# toc = time.perf_counter()

*Repeat with lidar_functions.add_str_to_filename...*

use if there are - in the output filename

In [ ]:
# # use if there are - in the output filename
# pathname = 'Piske_lidar/MRB/Merced_lidar/ASO/ASO_MRB_20210429/tindex/tiles/'
# for files in [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]:
#     target = files.replace("-","")
#     target = pathname + target
#     full_path = pathname+files
#     os.rename(full_path,target)

## Save tile boundaries
Save sqlite (shp) of tile boundaries of retiles <br>
Alter filepaths 

In [ ]:
# # save the tile index of a file to a new folder

# # input - full lidar/sqlite file patsh (i.e. 'lidar/lidar_files/filename.laz'), [str, str]
# def create_tindex(input_path, output_path):
#     boundary_cmd = ['pdal', 'tindex', 'create', '--tindex', output_path, '--filespec', input_path, '-f', 'SQLite']
#     subprocess.run(boundary_cmd)

**Parallel Processing**

In [ ]:
# # one folder
# tic = time.perf_counter()
# lidar_folder = 'path/to/lidar/files/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#         full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
#         # change depending on directory formats
#         output_path = [os.path.join(os.path.basename(lidar_folder),'tindex' + s[:-3] + 'sqlite') for s in onlyfiles]
#         executor.map(lidar_functions.create_tindex, full_path, output_path)
# toc = time.perf_counter()

## Copy files
Save a copy of files that meet criteria into a new folder <br>
The goal is to decrease processing time for datum conversions in the MRB <br>
Change file extents depending on watershed

In [ ]:
# # copy file based on llx and lly into output filename. Note that this function relies on the assumption that files follow the structure 

# # input_lid 'folder1/folder2/folder3a/filename.laz' and output_lid 'folder1/folder2/folder3b/filename.laz'
# # if files are held in a different file structure, code must be changed to accomodate change
# # input - full path to a lid file [str]
# # output_path - path to folder with ICB tiles (e.g. 'path/to/folder/') [str]
# def copy_lid_by_ext_ICB(full_path, output_path):
#     pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal command
#     pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
#     pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dict
#     # extract llx,lly of tile
#     minx = pdal_info_dict['metadata']['minx']
#     miny = pdal_info_dict['metadata']['miny']
#     maxx = pdal_info_dict['metadata']['maxx']
#     maxy = pdal_info_dict['metadata']['maxy']
#     # if file origin is within bounds of ICB extents, copy the file
#     if (minx <= 288000 and maxx >= 265000):
#         if (maxy >= 4165000 and miny<= 4180000):
#             input_lid = full_path
#             output_lid = output_path + os.path.basename(full_path)
#             pdal_copy_cmd = ['pdal','translate', input_lid, output_lid]
#             subprocess.run(pdal_copy_cmd)

**Parallel Processing**

All folders called "retile"

In [ ]:
# tic = time.perf_counter()
# all_folders = [x[0] for x in os.walk('path/to/directory/')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
# # save only those files 
# retile_list = [all_folders[i] for i in index_pos_list]
# for lidar_folder in retile_list:
#     if __name__ == '__main__':
#         with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#             onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#             full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
#             executor.map(copy_by_ext_ICB, full_path)
# toc = time.perf_counter()

## Datum Conversions

### Horizontal Datum Conversion
From PDAL library: https://pdal.io/tutorial/grid-shift.html

In [ ]:
horizontal_datum_conversion = json_base_path+'horizontal_datum_conversion.json'

In [ ]:
# input_las = 'path/to/filename/filename.las'
# output_las = "path/to/filename/filename.las"

In [ ]:
# reader_dict = {"type":"readers.las",
#               "filename":input_reproj_hor}

# reproject_dict = {"type":"filters.reprojection",
#                   "in_srs":"+proj=utm +zone=10 +datum=WGS84 +units=m +no_defs",
#                   "out_srs":"EPSG:26910"} #"in_srs":"EPSG:8999"
# writer_dict = {"type":"writers.las",
#                "a_srs":"EPSG:26910",
#               #  "scale_x":0.00000001,
#               #  "scale_y":0.00000001,
#               # "offset_x":"auto",
#               # "offset_y":"auto",
#               "filename":output_reproj_hor}

# pipeline_list = [reader_dict, reproject_dict, writer_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(horizontal_datum_conversion, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

### Vertical Datum Conversion
using output of code above

In [ ]:
# # best option for now 
# input_las = 'path/to/filename.las'
# output_las = 'path/to/filename.las'
# gtx_path = 'path/to/fielname.gtx' # /cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx
# reproj_fil_in_srs = '--filters.reprojection.in_srs=EPSG:26910+5703'
# reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:26910' + ' +geoidgrids='+gtx_path + ' +t_epoch=2010.0'
# writers_compression = '--writers.las.compression=true'
# writers_a_srs = '--writers.las.a_srs=+proj=utm +zone=10 +ellps=GRS80 +datum=NAD83 +units=m +no_defs' 

# Rasterize
Set up the rasterization pipeline which we will use throughout the workflow. <br>
We use two pipelines here, one which takes the mean of all points in a 1m pixel, using a 0.7m radius. <br>
*Note that this step is now incorporated into many of the below pipelines*

In [10]:
# name JSON file
rasterize_json_mean = json_base_path+'rasterize_mean.json'
rasterize_json_count = json_base_path+'rasterize_count.json'
rasterize_json_max = json_base_path+'rasterize_max.json'

In [11]:
# create a pipeline and save to a json file 
reader_dict = {'type':'readers.las'}
writers_gdal= {"type": "writers.gdal",
              'output_type': 'max',
              'resolution': '1.0',
              'radius': '0.7'}#,
             #'window_size':3}

pipeline_list = [reader_dict,writers_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(rasterize_json_max, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [12]:
tic = time.perf_counter()
input_path = 'MRB/Merced_lidar/NCALM/NCALM_MRB_20180921/HAG/'
output_path = 'MRB/Merced_lidar/NCALM/NCALM_MRB_20180921/CHM/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + s for s in onlyfiles]
        full_output_tif = [output_path + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(lidar_functions.rasterize_max, full_input_las, full_output_tif) 
toc = time.perf_counter()

**Single Files**

In [ ]:
# # Single Files
# reader = '--readers.las.filename='+'path/to/file/filename.las'
# writer = '--writers.gdal.filename='+'path/to/file/filename.tif'
# rasterize_command = ['pdal', 'pipeline', rasterize_json_mean, writer, reader]
# subprocess.run(rasterize_command)

**parallel processing**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/'
# output_path = 'path/to/tif/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_input_las = [input_path + s for s in onlyfiles]
#         full_output_tif = [output_path + s[:-4] + '.tif' for s in onlyfiles]
#         executor.map(lidar_functionsrasterize_count, full_input_las, full_output_tif) 
# toc = time.perf_counter()

# Ground Filter

Filter classified las files based on [Las 1.4 Specifications](http://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf)<br>
Where Classification 2 = ground <br>
Classification 7 = noise

In [ ]:
gf_json = json_base_path+'ground_filter_preClassified.json' # define path to json files

filter_range = {"type":"filters.range", 
                "limits":"Classification[2:2]"}
pipeline_list = [filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(gf_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single File**

In [ ]:
# input_las = 'path/to/lid/file/filenam.las'
# output_las = 'path/to/lid/file/filename.las'

In [ ]:
# range_cmd = ['pdal', 'translate', input_las,  output_las, '--json',gf_json]
# subprocess.run(range_cmd)

**All Files in Directory**

In [ ]:
# #329.1668573822826s
# tic = time.perf_counter()
# input_path = 'path/to/input/directory'
# output_path = 'path/to/output/directory'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + '/' + s for s in onlyfiles]
#         output_path_full = [output_path + '/' + s for s in onlyfiles]
#         executor.map(lidar_functions.ground_filter_preClassified, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

### Ground Classification

In [ ]:

filters_assign = {"type":"filters.assign",
                  "assignment":"Classification[:]=0"}
filters_elm = {'type':'filters.elm'}
filters_outlier = {'type':'filters.outlier'}
filters_smrf = {"type":"filters.smrf",
                "last":true,
                "ignore":"Classification[7:7]",
                "slope":0.2,
                "window":16,
                "threshold":0.45,
                "scalar":1.2}
{
      "type":"filters.elm"
    },
    {
      "type":"filters.outlier"
    },
    {
      "type":"filters.smrf",
      "last":true,
      "ignore":"Classification[7:7]",
      "slope":0.2,
      "window":16,
      "threshold":0.45,
      "scalar":1.2
    },
    {
      "type":"filters.range",
      "limits":"Classification[2:2]"
    }
  ]
}

## Create DEM
This pipeline combines aboves steps to avoid issues with merging rasters. We were seeing tile signatures when we merged all tif files after ground-filtering/rasterization, so here we combine these steps through merging the las and then rasterizing (importantly, we are not writing the merged las files to a new merged file, which defeatst the purpose of tiles). A weakness of this pipeline is that it doesn't allow for tile-level parallelization <br> see: https://pdal.io/pipeline.html

In [6]:
DEM_json = json_base_path+'DEM_from_las.json' # define path to json files

Define the las files we want to create a DEM from.

In [7]:
# input_path = 'path/to/gf/las/'
# output_tif = 'path/to/dem/filenamem.tif'
# onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))] # make a list of all filenames in directory
# input_list = [input_path + s for s in onlyfiles] # make a list of full filename paths in directory

Create the reader stages of the pipeline. Each file is read as an individual reader stage here, and we cheat here by copying the formatting of json files and creating a dictionary with the values as the correctly formatted inputs.

In [8]:
# filename_dict = {} # initiate an empty dict to hold the readers
# tags = ['']*len(input_list) # initiate an empty list, size = number of files
# filenames = ['']*len(input_list) # repeat
# for i in range(len(input_list)): # for each file, create a dictionary element with the values matching json formatting for file merging
#     filename_dict['filename_'+str(i)] = {'filename':input_list[i], 'tag':'A_'+str(i)}
#     tags[i] = 'A_'+str(i) # add a tag to the reader stage
#     filenames[i] = filename_dict[list(filename_dict)[i]] # Add all values to a list

Define the filter and writer stages of the pipeline

In [ ]:

# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# filter out the ground points of the tiles
filter_range = {"type":"filters.range",
                "limits":"Classification[2:2]"}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7',
               'window_size':3, # we want more of a wall-to-wall product here so we use a secondary algorithm to increase calculation distance
               'filename': output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
#pipeline_list.append(filter_range)
pipeline_list.append(writers_gdal)
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(DEM_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [9]:
# pdal_cmd = ['pdal','pipeline', DEM_json]
# subprocess.run(pdal_cmd)

# HAG and Noise Filter
Height above ground DEM (raster format). 

## Replace Z with HAG

In [ ]:
# define json path 
HAG_json = json_base_path + 'HAG_dem.json'

In [ ]:
# convert all z values to the height above ground 
target_dem = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/NCALM_2014_BE.tif'
filter_hag = {"type":"filters.hag_dem",
              "raster":target_dem, # full file path of target DEM (.tif)
              "zero_ground":"false"} # Do not assign 0 to ground classified points
filter_ferry = {"type":"filters.ferry",
                "dimensions":"HeightAboveGround=>Z"} # replace all Z dimensions with HAG instead of elevation
filter_range = {"type":"filters.range",
                "limits":"Z[-0.2:70]"} # apply a noise filter
pipeline_list = [filter_hag, filter_ferry,filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(HAG_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# input_las = "path/to/full/filename/filename.laz"
# output_las = "path/to/full/filename/filename.laz"
# HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
# subprocess.run(HAG_cmd)

**Multiple Files**

In [ ]:
# # multiple files
# pathname = "path/to/lid/folder/"
# output_pathname = "path/to/lid/folder/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
#     subprocess.run(HAG_cmd)

**Parallel Processing**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s for s in onlyfiles]
#         executor.map(lidar_functions.HAG_dem, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

## Add a new dimension
Add a new dimension to the file but maintain Z values

In [ ]:
# # this works to add a new dimension
# output_las_hag = 'input/lid/files/lidar.laz'
# filter_dem = '--filters.hag_dem.raster=' + target_DEM
# hag_addDim = ['pdal', 'translate', input_las_hag, output_las_hag, 'hag_dem' ,filter_dem, '--writers.las.extra_dims=HeightAboveGround=float32']
# subprocess.run(hag_addDim)

# Vertical Bias Correction

## Move-Merge_Clip
1. all vbc values <br>
2. vbc values <=5 <br>
3. vbc values <=3

### Move files
Make a copy of road covered tiles. <br>
The spatial extents of the road polygon are 738382,4371452 : 738686,4372063 <br>
chose only files with a min x > 737000 and a min y > 4370000

In [ ]:
# # defining source and destination
# # paths
# src = 'path/to/source/folder/*.la*'
# trg = 'path/to/target/folder/target'
# file_paths = glob.glob(src)

# for files in file_paths:
#     filename = os.path.basename(files)
#     if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
#     # copying the files to the
#     # destination directory
#         shutil.copy2(files, trg)

### Merge
Merge files over the road

In [ ]:
# input_path = 'path/to/target_lid/*.la*' # define path of input files
# output_fname = 'path/to/target/filename.laz'# set output filename
# input_fname = glob.glob(input_path) # save to list
# pdal_merge_command = input_fname
# pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
# #pdal_merge_command.insert(0,'-f')
# pdal_merge_command.insert(0,'merge')
# pdal_merge_command.insert(0,'pdal')
# subprocess.run(pdal_merge_command)

### Clip
Crop file to outline of the road shapefile

In [76]:
# name JSON file
clip_json = json_base_path +'clip_to_polygon.json'

In [82]:
# path_to_shapefile = 'SCB/supporting_files/bounding_box/hwy89_poly.shp'

In [83]:
# # extract wkt from hwy 89 polygon
# hwy89 = shapefile.Reader(path_to_shapefile)
# geom=[]
# for s in hwy89.shapes():
#     geom.append(pygeoif.geometry.as_shape(s)) 
# poly_base = pygeoif.MultiPolygon(geom)

In [84]:
# # create a pipeline and save to a json file 
# filter_crop = {'type':'filters.crop',
#                'polygon':poly_base.wkt}
# # write merged las to raster
# writers_gdal = {"type":"writers.text",
#                 "format":"csv",
#                 "order":"Z",
#                 'keep_unspecified':False}
# pipeline_list = [filter_crop]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(clip_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# writer_cmd = '--writers.text.filename='+output_txt

In [ ]:
# pdal_commands = ['pdal', 'translate', input_las, '--json', clip_json,writer_cmd]
# subprocess.run(pdal_commands)

### Combined Pipeline Method
Note that this example uses points instead of polygons. Alter accordingly. <br>
target_point should look something like "POINT(easting northing)"

In [ ]:
clip_json = json_base_path +'extract_las_atPoint.json'
filenames, tags = lidar_functions.create_command_template(input_path)

In [ ]:
def extract_las_atPoint(filenames, tags, target_point, output_txt,distance):
    # merge all las files or stages
    filter_merge = {"type":"filters.merge",
                    "tag": "merged",
                    "inputs": tags}
    # crop
    filter_crop = {'type':'filters.crop',
                   'point':target_point,
                   'distance':distance,
                   'inputs':'merged',
                   'tag': 'cropped'}
    # write merged las to raster
    writers_gdal = {"type":"writers.text",
                    "format":"csv",
                    "order":"Z",
                    "write_header":False,
                    'keep_unspecified':False,
                    'filename':output_txt}
    # Append each stage to a list prior to saving to json 
    pipeline_list = filenames.copy()
    pipeline_list.append(filter_merge)
    pipeline_list.append(filter_crop)
    pipeline_list.append(writers_gdal)

    pipeline_dict = {'pipeline' : pipeline_list}
    # save to json
    with open(extract_las_atPoint_json, 'w') as out:
        json.dump(pipeline_dict, out, indent=4)
    pdal_cmd = ['pdal','pipeline', extract_las_atPoint_json]
    subprocess.run(pdal_cmd)

## Calculate Stats Over Control Area
In Kostadinov et al., 2019 the vertical bias was corrected using the lowest 10th percentile value between the snow on and snow off flights above the road. We will calculate a number of statistics

### Calculate Stats

In [91]:
# # input_las - HAG, las file clipped to the road
# # output_path - path to output files
# # base_name - string, typically flight name
# def calculate_vertical_bias(input_las):
#     # convert height only to txt file
#     output_las_txt = input_las[:-3]+'csv'
#     txt_cmd = ['pdal', 'translate', input_las, output_las_txt, '-w', 'writers.text', '--writers.text.format=csv','--writers.text.order=Z', '--writers.text.keep_unspecified=false']
#     subprocess.run(txt_cmd)
#     # calculate stats
#     hag_arr = np.loadtxt(output_las_txt,skiprows=1)
#     lowest_10th_per = np.nanpercentile(hag_arr, 10)
#     mean_hag = np.nanmean(hag_arr)
#     median_hag = np.nanmedian(hag_arr)
#     stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
#     return(stats)

## Correct Z Values
Using PDAL filters.assign we can add a value to each lidar point
See lidar_functions.py - correct_by-target_val <br>

### Correct LAS Only
This involves simple command line function operations instead of a json derived pipeline. 

In [ ]:
# input_lid = 'path/to/input/filename/filename.laz'
# output_lid = 'path/to/output/file/filename.laz'
# filters_assign = '--filters.assign.value=Z=Z'+target_val
# assign_cmd = ['pdal', 'translate', input_lid, output_lid, 'assign' ,filters_assign]
# subprocess.run(assign_cmd)

In [ ]:
# # or use function
# # assign target val
# correct_by_targetVal_pipeline(target_val)
# full_input_path = 'path/to/input/filename/filename.laz'
# full_output_path = 'path/to/output/file/filename.laz'
# target_val = 0
# lidar_functions.correct_by_targetVal(full_input_path, full_output_path, target_val)

**Parallelize**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s for s in onlyfiles]
#         target_vals = np.repeat(target_val, len(full_path))
#         executor.map(lidar_functions.correct_by_targetVal, full_path, output_path_full, target_vals) 
# toc = time.perf_counter()

In [ ]:
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/corrected_las/'
target_val = 'Z+'+str(abs(NCALM_2014_hwy89_stats[1])) # lowest 10th percentile 
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path + s for s in onlyfiles]
        target_vals = np.repeat(target_val, len(full_path))
        executor.map(lidar_functions.correct_by_targetVal, full_path, output_path_full, target_vals)
toc = time.perf_counter()

In [ ]:
lidar_functions.correct_by_targetVal(full_path[0], output_path_full[0], target_vals[0])

In [ ]:
lidar_functions.correct_by_targetVal(full_path[0], output_path_full[0], target_val)

In [ ]:
filters_assign = '--filters.assign.value=Z=Z+0.08'
assign_cmd = ['pdal', 'translate', full_path[0], output_path_full[0], 'assign' ,filters_assign]
subprocess.run(assign_cmd)

### Correct LAS and Rasterize
For this section, we overwrite the json for each flight depending on the target value. This is beneficial for saving memory (no redundant corrected las files created)

In [185]:
correct_rasterize_json = json_base_path + 'correct_by_targetVal_rasterize.json'

In [65]:
correct_merge_rasterize_json = json_base_path + 'correct_merge_rasterize.json'

In [ ]:
# note that the corrected Z is Z - target val, because we are using the height above ground
# input: target_assign - str e.g 'Z+0.08'
def correct_by_targetVal_pipeline(target_assign):
    readers_las = {'type':'readers.las'}
    filters_assign = {'type': 'filters.assign',
                      'value':"Z="+target_assign}
    writers_gdal= {"type": "writers.gdal",
                  'output_type': 'mean',
                  'resolution': '1.0',
                  'radius': '0.7'}
    pipeline_list = [readers_las,filters_assign,writers_gdal]
    pipeline_dict = {'pipeline' : pipeline_list}
    with open(correct_rasterize_json, 'w') as out:
        json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# full_input_path = 'path/to/input/lid/filename.las'
# full_output_path = 'path/to/output/lid/filename.las'
# target_val = -9999
# correct_by_targetVal_pipeline(target_val)
# lidar_functions.correct_by_targetVal_rasterize(full_input_path, full_output_path)

**Parallel Processing**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/HAG/'
# output_path = 'path/to/output/folder/'
# target_val = ASO_20160518_hwy89_stats[1] # lowest 10th percentile 
# correct_by_targetVal_pipeline(target_val)
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s[:-3] +'tif' for s in onlyfiles]
#         executor.map(lidar_functions.correct_by_targetVal_rasterize, full_path, output_path_full)
# toc = time.perf_counter()

#### Combined Pipeline Method

In [7]:
# input_path = 'path/to/lidar/files/'
# output_tif = 'path/to/output/tif/filename.tif'
# onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))] # make a list of all filenames in directory
# input_list = [input_path + s for s in onlyfiles] # make a list of full filename paths in directory

Create the reader stages of the pipeline. Each file is read as an individual reader stage here, and we cheat here by copying the formatting of json files and creating a dictionary with the values as the correctly formatted inputs.

In [8]:
# filename_dict = {} # initiate an empty dict to hold the readers
# tags = ['']*len(input_list) # initiate an empty list, size = number of files
# filenames = ['']*len(input_list) # repeat
# for i in range(len(input_list)): # for each file, create a dictionary element with the values matching json formatting for file merging
#     filename_dict['filename_'+str(i)] = {'filename':input_list[i], 'tag':'A_'+str(i)}
#     tags[i] = 'A_'+str(i) # add a tag to the reader stage
#     filenames[i] = filename_dict[list(filename_dict)[i]] # Add all values to a list

Define the filter and writer stages of the pipeline

In [66]:
# # merge all las files or stages
# filter_merge = {"type":"filters.merge",
#                "tag": "merged",
#                "inputs": tags}
# # filter out the ground points of the tiles
# filter_assign = {'type': 'filters.assign',
#                  'value':"Z="+target_assign,
#                  'inputs':'merged',
#                  'tag':'corrected'}
# # write merged las to raster
# writers_gdal= {"type": "writers.gdal",
#                'output_type': 'mean',
#               'resolution': 1.0,
#               'radius': '0.7',
#                'bounds': writers_bounds,
#                'inputs': 'corrected',
#                'filename':output_tif}
# # Append each stage to a list prior to saving to json 
# pipeline_list = filenames.copy()
# pipeline_list.append(filter_merge)
# pipeline_list.append(filter_assign)
# pipeline_list.append(writers_gdal)

# pipeline_dict = {'pipeline' : pipeline_list}
# # save to json
# with open(correct_merge_rasterize_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

In [9]:
# tic = time.perf_counter()
# pdal_cmd = ['pdal','pipeline', correct_merge_rasterize_json]
# subprocess.run(pdal_cmd)
# toc = time.perf_counter()

# Calculate Canopy Metrics


Vegetation Height Strata:<br>
[-0.15:0.15), used to determine open sites<br>
[0.15:1.5), if there are returns in this region, we can look into weather there is short vegetation or understory here<br>
[1.5:3), we remove all pixels with returns here, classified as "low branches or significant shrub/grass/ground-veg"<br>
[3:), this is our threshold for "tall" vegetation<br>

## Vegetation Height Strata (counts)
Determine the number of returns (counts) of each height strata

In [ ]:
# This function allows us to change the json pipeline depending on the files
# the final output of a subprocess that uses this pipeline will be one raster with 1x1m pixels, each containing the number of returns (counts) of the specified lidar height strata
# e.g. if the range_limit is Z[-0.15:0.15), each pixel will give us the number of returns between -0.15 and 0.15 in elevation (HAG in this case)
# The goal of this function is to return a json pipeline with specific Z limits
# input: filenames - see: lidar_functions.create_command_template
# input: target_assign - if you need to perform a vertical bias correction on the snow-off flight, insert a string with the correction function (e.g. 'Z+0.05' if you need to add 0.05 to each lidar point), if no correction is necessary
# insert "Z+0" [str]
# input: writers_bounds - '([minx, maxx], [miny, maxy])';  e.g.'([730235.96, 738826.45], [4364741.66, 4372273.16])' [str]
# input: range_limits: Z limits for output file
def create_vegetation_heigh_strata(filenames, tags, target_assign, writers_bounds, range_limits, output_tif): 
    # merge all las files or stages
    filter_merge = {"type":"filters.merge",
                   "tag": "merged",
                   "inputs": tags}
    # bias correct files
    filter_assign = {'type': 'filters.assign',
                     'value':"Z="+target_assign,
                     'inputs':'merged',
                     'tag':'corrected'}
    # filter out the ground points of the tiles
    filter_range = {"type":"filters.range",
                    "limits":range_limits,
                   'inputs':'merged',
                   'tag':'filtered'}
    # write merged las to raster
    writers_gdal= {"type": "writers.gdal",
                   'output_type': 'count',
                  'resolution': '1.0',
                  'radius': '0.7',
                   'bounds': writers_bounds,
                   'inputs': 'filtered',
                   'filename':output_tif}
    # Append each stage to a list prior to saving to json 
    pipeline_list = filenames.copy()
    pipeline_list.append(filter_merge)
    pipeline_list.append(filter_assign)
    pipeline_list.append(filter_range)
    pipeline_list.append(writers_gdal)

    pipeline_dict = {'pipeline' : pipeline_list}
    return(pipeline_dict)

Constant inputs

In [ ]:
# these inputs remains the same for all heigh strata
input_path = 'path/to/HAG/files/'
filenames, tags = lidar_functions.create_command_template(input_path)
target_assign = "Z+0" # default
writers_bounds = '([minx,maxx],[miny,maxy])'

Variable inputs

[-0.15:0.15)

In [ ]:
# specify name of the json pipeline file
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_neg0pt15_0pt15.json'
# specify output tif file path
output_tif = 'pat/to/canopy_metric/folder/veg_height_strata/vegStrata_neg0pt15_0pt1.tif'
# heigh strata range limits
range_limits = 'Z[-0.15:0.15)'
# create pipline
pipeline_dict = create_vegetation_heigh_strata(filenames, tags, target_assign, writers_bounds, range_limits, output_tif)
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)

[0.15:1.5)

In [ ]:
# specify name of the json pipeline file
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_0pt15_1pt5.json'
# specify output tif file path
output_tif = 'pat/to/canopy_metric/folder/veg_height_strata/vegStrata_0pt15_1pt5.tif'
# heigh strata range limits
range_limits = 'Z[0.15:1.5)'
# create pipline
pipeline_dict = create_vegetation_heigh_strata(filenames, tags, target_assign, writers_bounds, range_limits, output_tif)
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)

[1.5:3)

In [ ]:
# specify name of the json pipeline file
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_1pt5_3.json'
# specify output tif file path
output_tif = 'pat/to/canopy_metric/folder/veg_height_strata/vegStrata_1pt5_3.tif'
# heigh strata range limits
range_limits = 'Z[1.5:3)'
# create pipline
pipeline_dict = create_vegetation_heigh_strata(filenames, tags, target_assign, writers_bounds, range_limits, output_tif)
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)

[3:)

In [ ]:
# specify name of the json pipeline file
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_3.json'
# specify output tif file path
output_tif = 'pat/to/canopy_metric/folder/veg_height_strata/vegStrata_3.tif'
# heigh strata range limits
range_limits = 'Z[1.5:3)'
# create pipline
pipeline_dict = create_vegetation_heigh_strata(filenames, tags, target_assign, writers_bounds, range_limits, output_tif)
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)

## Old Code

[-0.15,0.15)

In [ ]:
# range_json_neg0pt15_0pt15 = json_base_path+'filter_pts_neg0pt15_0pt15.json'
# reader_dict = {'type':'readers.las'}
# filter_range_neg0pt15_0pt15 = {"type":"filters.range",
#                 "limits":"Z[-0.15:0.15)"}
# writers_gdal_count= {"type": "writers.gdal",
#               'output_type': 'count',
#               'resolution': '1.0',
#               'radius': '0.7',
#               'window_size':2}
# pipeline_list = [reader_dict,filter_range_neg0pt15_0pt15,writers_gdal_count]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(range_json_neg0pt15_0pt15, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

[0.15,2)

In [ ]:
# range_json_0pt15_2 = json_base_path+'filter_pts_0pt15_2.json'
# reader_dict = {'type':'readers.las'}
# filter_range_0pt15_2 = {"type":"filters.range",
#                 "limits":"Z[0.15:2)"}
# writers_gdal_count= {"type": "writers.gdal",
#               'output_type': 'count',
#               'resolution': '1.0',
#               'radius': '0.7'}
# pipeline_list = [reader_dict,filter_range_0pt15_2,writers_gdal_count]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(range_json_0pt15_2, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

[2,inf)

In [ ]:
# range_json_2 = json_base_path+'filter_pts_2.json'
# reader_dict = {'type':'readers.las'}
# filter_range_2 = {"type":"filters.range",
#                 "limits":"Z[2:)"}
# writers_gdal_count= {"type": "writers.gdal",
#               'output_type': 'count',
#               'resolution': '1.0',
#               'radius': '0.7'}
# pipeline_list = [reader_dict,filter_range_2,writers_gdal_count]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(range_json_2, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

[2,inf) Ground

In [ ]:
# range_json_2_ground = json_base_path+'filter_pts_2_ground.json'
# reader_dict = {'type':'readers.las'}
# filter_range_2_ground = {"type":"filters.range",
#                 "limits":"Z[2:), Classification[2:2]"}
# writers_gdal= {"type": "writers.gdal",
#               'output_type': 'count',
#               'resolution': '1.0',
#               'radius': '0.7'}
# pipeline_list = [reader_dict,filter_range_2_ground,writers_gdal]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(range_json_2_ground, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

[2,inf) Nonground

In [ ]:
# range_json_2_nonground = json_base_path+'filter_pts_2_nonground.json'
# reader_dict = {'type':'readers.las'}
# filter_range_2_nonground = {"type":"filters.range",
#                 "limits":"Z[2:), Classification![2:2]"}
# writers_gdal= {"type": "writers.gdal",
#               'output_type': 'count',
#               'resolution': '1.0',
#               'radius': '0.7'}
# pipeline_list = [reader_dict,filter_range_2_nonground,writers_gdal]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(range_json_2_nonground, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

**Single File**

In [ ]:
# input_las = 'path/to/input/lid/filename.las'
# output_las = 'path/to/output/lid/filename.las'
# strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',range_json]
# subprocess.run(strata_cmd)

**All Files in Directory**

In [ ]:
# tic = time.perf_counter()
# pathname = "path/to/lid/folder/"
# output_pathname = 'path/to/lid/folder/'
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',range_json]
#     subprocess.run(strata_cmd)
# toc = time.perf_counter()

**Parallelization**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s for s in onlyfiles]
#         executor.map(lidar_functions.range_json, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

### Clipping Geometries
The goal of this portion of the code is to clip the raster based on the control areas (in the case of SCB, hwy 89)<br>
See: https://pdal.io/tutorial/clipping/index.html#clipping

In [ ]:
[
    "autzen.laz",
    {
      "type":"filters.overlay",
      "dimension":"Classification",
      "datasource":"attributes.vrt",
      "layer":"OGRGeoJSON",
      "column":"CLS"
    },
    {
      "type":"filters.range",
      "limits":"Classification[5:5]"
    },
    "output.las"
]

In [ ]:
# name JSON file
clipping_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/clip_las_to_shp.json'

filter_overlay_dict = {"type":"filters.overlay",
                       "dimension":"Classification",
                       "datasource":"SCB/bounding_box/hwy89_poly.shp",
                       "column":"OBJECTID"}
filter_range_dict = {"type":"filters.range",
                     "limits":"Classification[4193:4193]"}

pipeline_list = [filter_overlay_dict,filter_range_dict]
with open(clipping_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/reproj_NAD83_NAVD88/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
pdal_clip = ['pdal', 'translate', input_las, output_las, '--json', output_json]
subprocess.run(pdal_clip)

# ----------------------

# Applied

In [28]:
# define all json files
filterMergeRasterize_json = json_base_path + 'filterMergeRasterize.json'

## Pre-Processing

### Info

In [68]:
# define input file
input_lid = r'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/CA_20210429_pN94S_f1_tiledlas_fl195341_ch1_ti-6720_tj-4500_ts60_v2_UTMZ11.laz'
pdal_info_cmd = ['pdal','info',input_lid] # general info
pdal_metadata_cmd = ['pdal','info',input_lid,'--metadata'] # full file metadata, including details crs
subprocess.run(pdal_metadata_cmd)

{
  "file_size": 33332101,
  "filename": "MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/CA_20210429_pN94S_f1_tiledlas_fl195341_ch1_ti-6720_tj-4500_ts60_v2_UTMZ11.laz",
  "metadata":
  {
    "comp_spatialreference": "PROJCS[\"WGS 84 / UTM zone 11N\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"latitude_of_origin\",0],PARAMETER[\"central_meridian\",-117],PARAMETER[\"scale_factor\",0.9996],PARAMETER[\"false_easting\",500000],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH]]",
    "compressed": true,
    "count": 5712854,
    "creation_doy": 76,
    "creation_year": 2022,
    "dataformat_id": 6,
    "dataoffset": 1725,
    "filesour

CompletedProcess(args=['pdal', 'info', 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/CA_20210429_pN94S_f1_tiledlas_fl195341_ch1_ti-6720_tj-4500_ts60_v2_UTMZ11.laz', '--metadata'], returncode=0)

In [69]:
pdal_info_results = subprocess.run(pdal_metadata_cmd, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # create dict with metadata info
#pdal_info_dict # print results

In [70]:
# pdal_info_dict

Check Extents

In [71]:
lidar_folder = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/NAD83_NAD83_epoch2010/'
# lidar_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
full_paths = [lidar_folder +f for f in onlyfiles]

In [72]:
# NCALM_2014_ext = lidar_functions.check_flight_extent(full_paths)
# ASO_SCB_20160326_ext = lidar_functions.check_flight_extent(full_paths)
# ASO_SCB_20160417_ext = lidar_functions.check_flight_extent(full_paths)
ASO_SCB_20160518_ext = lidar_functions.check_flight_extent(full_paths)

In [73]:
# print(NCALM_2014_ext)
# print(ASO_SCB_20160326_ext)
# print(ASO_SCB_20160417_ext)
print(ASO_SCB_20160518_ext)

[730235.96, 738826.45, 4364741.66, 4372273.16, 8590.48999999999, 7531.5]


### Retile

In [ ]:
lidar_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
retile_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/retile_uo/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
for files in onlyfiles:
    full_path = os.path.join(lidar_folder, files)
    output_path = retile_folder+'#' + files
    retile_command = ['pdal', 'tile', full_path, output_path, '--length=1000','--buffer=50']
    subprocess.run(retile_command)

### Rename

In [ ]:
lidar_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
tic = time.perf_counter()
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
        onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
        full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
        executor.map(lidar_functions.rename_llx_lly_b, full_path) #running 10 times
toc = time.perf_counter()

### Save Tile Boundaries

In [ ]:
# one folder
tic = time.perf_counter()
lidar_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/'
output_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/tindex/original/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
        full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
        # change depending on directory formats
        #output_path = [os.path.join(os.path.basename(lidar_folder),'tindex/tiles/' + s[:-3] + 'sqlite') for s in onlyfiles]
        output_path = [output_folder + s[:-3] + 'sqlite' for s in onlyfiles]
        executor.map(lidar_functions.create_tindex, full_path, output_path) #running 10 times
toc = time.perf_counter()

### Copy Files

In [ ]:
tic = time.perf_counter()
lidar_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/'
output_path = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/ICB_tiles/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
full_paths = [os.path.join(lidar_folder, s) for s in onlyfiles] 
output_paths = np.repeat(output_path, len(full_paths))
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        executor.map(lidar_functions.copy_lid_by_ext_ICB, full_paths, output_paths)
toc = time.perf_counter()

## Create DEM

In [17]:
DEM_json = json_base_path+'DEM_from_las.json' # define path to json files

In [18]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/NCALM_2014_BE.tif'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
input_list = [input_path + s for s in onlyfiles]

In [19]:
filename_dict = {}
tags = ['']*len(input_list)
filenames = ['']*len(input_list)
for i in range(len(input_list)):
    filename_dict['filename_'+str(i)] = {'filename':input_list[i], 'tag':'A_'+str(i)}
    tags[i] = 'A_'+str(i)
    filenames[i] = filename_dict[list(filename_dict)[i]]

In [20]:
# filter out the ground points of the tiles
filter_range = {"type":"filters.range",
                "limits":"Classification[2:2]"}
# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7',
               'window_size':3,
               'filename': output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
pipeline_list.append(filter_range)
pipeline_list.append(writers_gdal)
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(DEM_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
tic = time.perf_counter()
pdal_cmd = ['pdal','pipeline', DEM_json]
subprocess.run(pdal_cmd)
toc = time.perf_counter()

### Heigh Above Ground

In [ ]:
# parallel processing
# time = 1.1 min
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path  + s for s in onlyfiles]
        executor.map(lidar_functions.HAG_dem, full_path, output_path_full)

In [154]:
# parallel processing
# time = 1.1 min
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/NAD83_NAD83_epoch2010/'
output_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path  + s for s in onlyfiles]
        executor.map(lidar_functions.HAG_dem, full_path, output_path_full)

In [88]:
# parallel processing
# time = 1.1 min
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/NAD83_NAD83_epoch2010/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path  + s for s in onlyfiles]
        executor.map(lidar_functions.HAG_dem, full_path, output_path_full)

## Vertical Bias Correction

### Move Files

Applied

In [159]:
# defining source and destination
# paths
src = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/HAG/*.la*'
trg = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_lid'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

In [158]:
# defining source and destination
# paths
src = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/*.la*'
trg = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_lid'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

**NCALM 2008**

In [86]:
# extents extracted from QGIS
min_x = 731945
max_x = 734327
min_y = 4368517
max_y = 4375256

In [89]:
# defining source and destination
# paths
src = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/HAG/*.la*'
trg = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/target_lid'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) < max_x and int(files[-11:-4]) > min_y:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

### Merge

In [165]:
input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/hwy89_vertical_bias/target_lid/*.la*' # define path of input files
input_fname = glob.glob(input_path) # save to list
output_fname = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/hwy89_vertical_bias/target_lid/ASO_SCB_20160326_hwy89_merge.'+input_fname[0][-3:]# set output filename
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

In [122]:
# input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/target_lid/*.la*' # define path of input files
# input_fname = glob.glob(input_path) # save to list
# output_fname = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/target_lid/NCALM_SCB_20080210_merge.'+input_fname[0][-3:]# set output filename
# pdal_merge_command = input_fname
# pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
# #pdal_merge_command.insert(0,'-f')
# pdal_merge_command.insert(0,'merge')
# pdal_merge_command.insert(0,'pdal')
# subprocess.run(pdal_merge_command)

### Clip

#### ASO 2016

Applied

In [117]:
input_las = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/target_lid/ASO_SCB_20160518_hwy89_merge.las'
output_txt = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.txt'

In [119]:
pdal_commands = ['pdal', 'translate', input_las, output_txt, '--json', clip_json]
subprocess.run(pdal_commands)

#### NCALM 2008

In [50]:
# name JSON file
extract_las_atPoint = json_base_path +'extract_las_atPoint.json'

In [230]:
# path_to_shapefile = 'SCB/supporting_files/Harpold_data_paper/snotel_20080210_poly_b.shp'
# # extract wkt from hwy 89 polygon
# snotel_pts = shapefile.Reader(path_to_shapefile)
# snotel_pts_data = snotel_pts.records()
# geom=[]
# for s in snotel_pts.shapes():
#     geom.append(pygeoif.geometry.as_shape(s)) 
# poly_base = pygeoif.MultiPolygon(geom)
# snote_pts = None

In [44]:
# Read in ground_truthed data
def csv_to_list(input_csv):
    src = open(input_csv)
    csvreader = csv.reader(src)
    header = next(csvreader)
    output_list = []
    for row in csvreader:
        output_list.append(row)
    src.close()
    return(header, output_list)

In [45]:
SNOTEL_src = "SCB/supporting_files/Harpold_data_paper/snowdepth_filtered.csv"
snotel_header, snotel_data = csv_to_list(SNOTEL_src)
hunt_src = 'SCB/supporting_files/Huntingon_2008_snow/Hungington_2008_snow.csv'
hunt_header, hunt_data = csv_to_list(hunt_src)

In [54]:
snotel_data

[['Independence Creek',
  '540',
  '39.490053',
  '-120.282302',
  '131.1666667',
  '733715',
  '4374687'],
 ['Independence Camp',
  '539',
  '39.452701',
  '-120.29369',
  '140.0416667',
  '732860',
  '4370511'],
 ['Independence Lake',
  '541',
  '39.427512',
  '-120.313362',
  '212.2083333',
  '731251',
  '4367665']]

In [58]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/HAG/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/vertical_bias_correction/'
filenames, tags = lidar_functions.create_command_template(input_path)

In [63]:
output_txt = output_path+'snotel_'+snotel_data[0][1]+'.csv'
target_point = 'POINT('+snotel_data[0][5]+' '+snotel_data[0][6]+')'
#extract_last_atPoint(

In [64]:
target_point

'POINT(733715 4374687)'

In [216]:
# target_point = 'Point(731945.81 4368517.23)'
# # target_point = 'Point(732770.09 4370518.08)'
# output_txt = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/clipped/snotel_541.csv'
# # output_txt = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/clipped/snotel_539.csv'

In [51]:
def extract_las_atPoint(filenames, tags, target_point, output_txt):
    # merge all las files or stages
    filter_merge = {"type":"filters.merge",
                    "tag": "merged",
                    "inputs": tags}
    # crop
    filter_crop = {'type':'filters.crop',
                   'point':target_point,
                   'distance':1,
                   'inputs':'merged',
                   'tag': 'cropped'}
    # write merged las to raster
    writers_gdal = {"type":"writers.text",
                    "format":"csv",
                    "order":"Z",
                    'keep_unspecified':False,
                    'filename':output_txt}
    # Append each stage to a list prior to saving to json 
    pipeline_list = filenames.copy()
    pipeline_list.append(filter_merge)
    pipeline_list.append(filter_crop)
    pipeline_list.append(writers_gdal)

    pipeline_dict = {'pipeline' : pipeline_list}
    # save to json
    with open(extract_las_atPoint, 'w') as out:
        json.dump(pipeline_dict, out, indent=4)
    pdal_cmd = ['pdal','pipeline', extract_las_atPoint]
    subprocess.run(pdal_cmd)

In [217]:
# # merge all las files or stages
# filter_merge = {"type":"filters.merge",
#                "tag": "merged",
#                "inputs": tags}
# # crop
# filter_crop = {'type':'filters.crop',
#                  'point':target_point,
#                'distance':2,
#                'inputs':'merged',
#               'tag': 'cropped'}
# # write merged las to raster
# writers_gdal = {"type":"writers.text",
#                 "format":"csv",
#                 "order":"Z",
#                 'keep_unspecified':False,
#                'filename':output_txt}
# # Append each stage to a list prior to saving to json 
# pipeline_list = filenames.copy()
# pipeline_list.append(filter_merge)
# pipeline_list.append(filter_crop)
# pipeline_list.append(writers_gdal)

# pipeline_dict = {'pipeline' : pipeline_list}
# # save to json
# with open(clip_json_SNOTEL, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

In [218]:
pdal_cmd = ['pdal','pipeline', clip_json_SNOTEL]
subprocess.run(pdal_cmd)

CompletedProcess(args=['pdal', 'pipeline', 'piske_processing/PDAL_workflow/JSON/clip_to_geometries_snotel.json'], returncode=0)

### Calculate Stats

Applied
Using 2014 NCALM Flight

In [6]:
ASO_20160326_hwy89_stats = lidar_functions.calculate_vertical_bias('SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/hwy89_vertical_bias/clipped/ASO_SCB_20160326_hwy89_clip.laz')
ASO_20160417_hwy89_stats = lidar_functions.calculate_vertical_bias('SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/clipped/ASO_SCB_20160417_hwy89_clip.laz')
ASO_20160518_hwy89_stats = lidar_functions.calculate_vertical_bias('SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.las')

In [8]:
NCALM_2014_hwy89_stats = lidar_functions.calculate_vertical_bias('SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.las')

In [9]:
print(ASO_20160326_hwy89_stats)
print(ASO_20160417_hwy89_stats)
print(ASO_20160518_hwy89_stats)
print(NCALM_2014_hwy89_stats)

['lowest_10th', 0.24, 'mean', 0.3113292433537832, 'median', 0.31]
['lowest_10th', 0.27, 'mean', 0.3502506714413608, 'median', 0.35]
['lowest_10th', 0.37, 'mean', 0.4511492281303602, 'median', 0.44]
['lowest_10th', -0.05, 'mean', 0.02644937635008014, 'median', 0.02]


**NCALM 2008**

In [238]:
# input_las_txt = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/clipped/snotel_541.csv'
input_las_txt = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SNOTEL_vertical_bias/clipped/snotel_539.csv'

In [239]:
hag_arr = np.loadtxt(input_las_txt,skiprows=1)
lowest_10th_per = np.nanpercentile(hag_arr, 10)
mean_hag = np.nanmean(hag_arr)
median_hag = np.nanmedian(hag_arr)
stats_539 = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]

In [240]:
stats_539

['lowest_10th', 1.22, 'mean', 1.3270967741935482, 'median', 1.33]

In [246]:
SD_539_error = stats_539[1] - SD_539_true
# SD_541_error = stats_541[1] - SD_541_true

In [249]:
SD_539_error

-0.18041666700000003

### Correct and Rasterize
In this case we know that the May flight has the most limited extent. One way to check for this would be to use a similar code to 

In [19]:
correct_merge_rasterize_json = json_base_path + 'correct_merge_rasterize.json'

In [29]:
# input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/HAG/'
# filenames, tags = lidar_functions.create_command_template(input_path)
# output_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/corrected_tif/ASO_SCB_20160518_vbc.tif'
# target_assign = 'Z-'+str(abs(ASO_20160518_hwy89_stats[1]))

In [252]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/HAG/'
filenames, tags = lidar_functions.create_command_template(input_path)
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc.tif'
target_assign = 'Z+'+str(abs(SD_539_error))

Create the reader stages of the pipeline. Each file is read as an individual reader stage here, and we cheat here by copying the formatting of json files and creating a dictionary with the values as the correctly formatted inputs.

In [74]:
# [730235.96, 738826.45, 4364741.66, 4372273.16, 8590.48999999999, 7531.5]

writers_bounds = '(['+ str(ASO_SCB_20160518_ext[0])+','+ str(ASO_SCB_20160518_ext[1])+'],['+str(ASO_SCB_20160518_ext[2])+','+str(ASO_SCB_20160518_ext[3])+'])'

In [254]:
# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# filter out the ground points of the tiles
filter_assign = {'type': 'filters.assign',
                 'value':"Z="+target_assign,
                 'inputs':'merged',
                 'tag':'corrected'}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'mean',
              'resolution': 1.0,
              'radius': '0.7',
               'bounds': writers_bounds,
               'inputs': 'corrected',
               'filename':output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
pipeline_list.append(filter_assign)
pipeline_list.append(writers_gdal)

pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(correct_merge_rasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [255]:
# time = ~3 min
pdal_cmd = ['pdal','pipeline', correct_merge_rasterize_json]
subprocess.run(pdal_cmd)

CompletedProcess(args=['pdal', 'pipeline', 'piske_processing/PDAL_workflow/JSON/correct_merge_rasterize.json'], returncode=0)

## Snow-Off Processing

### Vegetation Strata

**Combined Pipeline Method**
We'll use the same bounds as above

In [79]:
#filterMergeRasterize_json = json_base_path+'filterMergeRasterize.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_neg0pt15_0pt15.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_0pt15_2.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_2.json'
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_neg0pt15_0pt15.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_2_nonground.json'


In [80]:
writers_bounds = '(['+ str(ASO_SCB_20160518_ext[0])+','+ str(ASO_SCB_20160518_ext[1])+'],['+str(ASO_SCB_20160518_ext[2])+','+str(ASO_SCB_20160518_ext[3])+'])'

In [81]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/vegStrata_neg0pt15_0pt1.tif'
filenames, tags = lidar_functions.create_command_template(input_path)

In [82]:
# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# filter out the ground points of the tiles
filter_range = {"type":"filters.range",
                "limits":"Z[-0.15:0.15)",
               'inputs':'merged',
               'tag':'filtered'}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7',
               'bounds': writers_bounds,
               'inputs': 'filtered',
               'filename':output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
pipeline_list.append(filter_range)
pipeline_list.append(writers_gdal)

pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [83]:
tic = time.perf_counter()

In [84]:
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)

CompletedProcess(args=['pdal', 'pipeline', 'piske_processing/PDAL_workflow/JSON/filterMergeRasterize_neg0pt15_0pt15.json'], returncode=0)

In [85]:
toc = time.perf_counter()

In [86]:
(toc-tic) / 60

7.017118186666661

**Move to Raster workflow**

# ------------------------------

# Extras

## File format conversions

### Convert .las to .txt
see: https://pdal.io/stages/writers.text.html

In [ ]:
# set up json file commands
output_txt = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000Test.asc'
output_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_txt.json'

# create a pipeline and save to a json file 

filter_dict = {'type':'readers.las',
               'override_srs': "EPSG:4326",
              'filename': input_las} # we are reading in a las file
rasterize_dict = {'type':'writers.las',
'format':'geojson',
'order':'X,Y,Z',
'keep_unspecified':'false',
'filename':output_txt}


pipeline_list = [filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
json_path = 'lidar_processing/python_scripts/PDAL/JSON/las_to_txt.json'
pdal_commands = ['pdal', 'pipeline', json_path]
subprocess.run(pdal_commands)

### .laz to .las

In [ ]:
# # set up json file commands
# input_laz = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.laz'
# output_las = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.las'
# output_json = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'

# # create a pipeline and save to a json file 

# filter_dict = {'type':'readers.las',
#                'filename': input_las} # we are reading in a las file
# translate_dict = {'type':'writers.las',
#                   "a_srs": "EPSG:4326",
#                   'filename':output_las}


# pipeline_list = [filter_dict, translate_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(output_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [ ]:
# json_path = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'
# pdal_commands = ['pdal', 'pipeline', json_path]
# subprocess.run(pdal_commands)

# Raster Caluclations

In [ ]:
for i in [0,1,2,3,4,5]:
    apr_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_00'+str(i)+'.tif'
    may_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
    output = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/raster_subtract/mcc_part_b_tile_004_00'+str(i)+'.tif'
    raster_sub = ['gdal_calc.py', '-a', apr_elev, '-b', may_elev, '--calc="a - b"', '--outfile', output]
    subprocess.run(raster_sub)

# Theo Code

In [ ]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

z_min = 0.15
z_max = 2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.tif'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_tif.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'GTiff',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

# z_min = 0.15
# z_max = 2
# z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.asc'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_asc.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'XYZ',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz

where path_to_laz_folder is the path to the LAS/LAZ file (you just need the folder path, not the file path).

:/input is the new folder that will be created in your Docker container that will hold your point cloud.

0b is just the image id of pdal

/input/test.laz is the path to the point cloud in the Docker container.


In [ ]:
path_to_laz_folder = 'lidar_processing/python_scripts/PDAL/test_las'


In [ ]:
sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz


In [ ]:
import json
import subprocess
import pdal

def assemblePipeline(input_las, list_of_dicts):
    pipeline_list = [input_las]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline' : pipeline_list}
    return pipeline_dict

def makeHeightFilter(height, buffer):
    z_min = height - buffer/2
    z_max = height + buffer/2
    z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
    heightDict = {'type':'filters.range', 'limits':z_range}
    return heightDict

def makeRasterizeFilter(output_raster, resolution, epsg):
    rasterize_dict = {'filename':output_raster,
                      'gdaldriver':'GTiff',
                      'output_type':'count',
                      'resolution':resolution,
                      'override_srs' : epsg,
                      'type': 'writers.gdal'}
    return rasterize_dict

def convertTifForPIL(input_raster, output_raster, epsg):
    ''' GDAL bindings are an alien concept to me, so I gave up and used
        subprocess.'''
    commands = ['gdal_translate', input_raster, output_raster, '-ot', 'Byte', '-a_srs', epsg]
    subprocess.run(commands)


def buildHeightSlice(input_las, height, buffer, output_raster, resolution, epsg, json_path=None):
    filter_dict = makeHeightFilter(height, buffer)
    rasterize_dict = makeRasterizeFilter(output_raster, resolution, epsg)
    filter_list = [filter_dict, rasterize_dict]
    pipeline_dict = assemblePipeline(input_las, filter_list)

    if json_path is not None:
        with open(json_path, 'w') as out:
            json.dump(pipeline_dict, out, indent=4)
        pdal_commands = ['pdal', 'pipeline', json_path]
        subprocess.run(pdal_commands)
    else:
        pdal_commands = json.dumps(pipeline_dict)
        pipeline = pdal.Pipeline(pdal_commands)
        pipeline.execute()

input_las = '/Users/theo/data/las/TLS_0244_20180612_01_v003_30m_clip_height_norm.las'
height = 1.37
buffer = 0.05
z_min = height - buffer/2
z_max = height + buffer/2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
temp_raster = '/Users/theo/Pictures/almost_cool.tif'
final_raster = '/Users/theo/Pictures/cool.tif'
resolution = 0.01
epsg = 'EPSG:3310'

buildHeightSlice(input_las, height, buffer, temp_raster, resolution, epsg)
convertTifForPIL(temp_raster, final_raster, epsg)

In [ ]:
import json
import subprocess
import pdal
import argparse

# Create flags for the user to utilize.
parser = argparse.ArgumentParser(description="Generate JSON pipeline to generate DTM from a point cloud.")
      
required = parser.add_argument_group('Required arguments')
required.add_argument('-crs', '--coordinate_system', required=True, action='store', help="EPSG code.")
required.add_argument('-i', '--infile', required=True, action='store', help="Input path to point cloud")
required.add_argument('-o', '--outfile', required=True, action='store', help="Output path.")
args = parser.parse_args()

def generateJSON(infile, list_of_dicts):
    pipeline_list = [infile]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline': pipeline_list}
    with open("pipeline.json", 'w') as out:
        json.dump(pipeline_dict, out, indent=4)

def generateDTM(epsg, infile, outfile):
    reproject_dict = {"type": "filters.reprojection",
                      "out_srs": "EPSG:{}".format(epsg)}
    reclassify_zero_dict = {"type": "filters.assign",
                       "assignment": "Classification[:]=0"}
    elm_dict = {"type": "filters.elm"}
    outlier_dict = {"type": "filters.outlier"}
    smrf_dict = {"type": "filters.smrf", "ignore": "Classification[7:7]",
                 "slope": 0.2, "window": 16, "threshold": 0.45, "scalar": 1.2}
    range_dict = {"type":"filters.range", "limits":"Classification[2:2]"}
    output_dict = {"filename": outfile, "gdaldriver": "GTiff", "output_type": "all", "resolution": 0.01, "type": "writers.gdal"}
    list_of_dicts = list([reproject_dict, reclassify_zero_dict, elm_dict, outlier_dict, smrf_dict, range_dict, output_dict])
    generateJSON(infile, list_of_dicts)
    pdal_cmds = ['pdal', 'pipeline', 'pipeline.json']
    subprocess.run(pdal_cmds)
    
generateDTM(args.coordinate_system, args.infile, args.outfile)

# More helpful things

### Get stats of a dataset
see: https://www.spatialised.net/lidar-qa-with-pdal-part-1/

In [ ]:
# name JSON file

stats_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/stats.json'

In [ ]:
reader_las = {"type":"readers.las",
              "filename": input_las_stats}
filter_stats = {"type":"filters.stats",
                "dimensions":"Z",
                "global":"Z",
                "advanced":"true"}
pipeline_list = [reader_las, filter_stats]
#pipeline_dict = {reader_las, filter_stats}
# with open(stats_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [ ]:
pipeline_list

In [ ]:
pipeline = pdal.Pipeline(json.dumps(pipeline_list))
pipeline.execute()

In [ ]:
json.loads(pipeline.metadata)["metadata"]["filters.stats"]["statistic"]

## Navigating folders/files

**create list of files/folders with a wildcard (*)**

In [ ]:
# ex. list all files in folder4 that end in .laz = folder1/folder2/folder3/folder4/*.laz
# ex. list all folders named folder3 in folder 1 = foler1/*/folder3
# ex. list all list all contents in folder2 = folder2/* - note just folder 2, no subdirectories
glob_cmd = 'path'
glob_exe = glob.glob(file_glob_cmd)

**create a list with all directories/subdirectories on a path**

In [ ]:
all_folders = [x[0] for x in os.walk('path')]

**create a list with all directories/subdirectories on a path with specific folders**

In [ ]:
all_folders = [x[0] for x in os.walk('path')]
index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ] # must change this to meet requirements
full_list = [all_folders[i] for i in index_pos_list]

**get name of the directory just above one listed**

In [ ]:
subdirname = os.path.basename(os.path.dirname('path'))

**create a list with only filenames**

In [ ]:
onlyfiles = [f for f in os.listdir('path') if os.path.isfile(os.path.join('path', f))]

**create a list with full file paths**

In [ ]:
full_path = ['path' + '/' + s for s in onlyfiles]

## Parallel Processing

In [ ]:
# time = 142.5
tic = time.perf_counter()
list(map(function, args));
toc = time.perf_counter()

In [ ]:
# time = 53 s
tic = time.perf_counter()
if __name__ == "__main__":
    pool = Pool(3)
    pool.map(function, arg)
    pool.close()
toc = time.perf_counter()

In [ ]:
# time = very fast? .06s
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=3)
    executor.map(function, arg)
toc = time.perf_counter()

In [ ]:
# time = 0.22
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ProcessPoolExecutor(max_workers=3)
    executor.map(function, arg)
toc = time.perf_counter()